# s2pipe – Local launcher (Step 2: preprocess)

This notebook runs **Step 2 (preprocess)** locally on a small subset of the Step‑1 download output (the Step‑1 `index.json` manifest).

It demonstrates the recommended **two‑pass workflow** for dataset‑level normalization:

1. **Pass 1 — compute stats**: scan tiles and build per‑band histograms to estimate clipping percentiles and compute mean/std **after clipping**  
   (`normalize.mode="compute_only"`). This produces `stats.json` (and optionally `stats.npz`).

2. **Pass 2 — apply stats**: re-run preprocessing, apply percentile clipping and per‑band normalization on valid pixels, and export tiles  
   (`normalize.mode="apply_with_stats"`).

This notebook also enables **TOA reflectance** conversion for L1C inputs (`to_toa_reflectance=True`).

### What you will get
- `x.tif` — preprocessed band stack (C,H,W) plus validity mask channel(s)
- `y.tif` — label raster derived from `SCL_20m`
- `angles.tif` — optional; exported separately on the native coarse angle grid (not appended to `x.tif`)
- `meta.json`, Step‑2 `index.json`, and the run manifest (`run=<id>.jsonl`)

Output is written under `<OUT_DIR>/processed/...` and `<OUT_DIR>/meta/step2/...`.


## 1) Environment setup and project discovery

The code below locates the repository root by walking up until it finds `pyproject.toml`.
It then makes the package importable either by:
- installing it in editable mode (`pip install -e .`), or
- adding `src/` to `sys.path` (useful for quick local experiments).

If you switch between the two approaches, restarting the kernel is recommended.


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
while (
    not (PROJECT_ROOT / "pyproject.toml").exists()
    and PROJECT_ROOT != PROJECT_ROOT.parent
):
    PROJECT_ROOT = PROJECT_ROOT.parent

print("PROJECT_ROOT:", PROJECT_ROOT)

# If imports fail, choose one of the options below:
# !pip install -e .

# Alternative without installation (add src/ to sys.path):
src_dir = PROJECT_ROOT / "src"
if src_dir.exists() and str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))
print("sys.path[0]:", sys.path[0])

## 2) Configure paths to Step‑1 `index.json` and the output directory

- `INDEX_PATH` must point to the Step‑1 `index.json` produced by the download step.
- `OUT_DIR` is the dataset root. Step‑2 will create:
  - `<OUT_DIR>/processed/...` for rasters (`x.tif`, `y.tif`, optional `angles.tif`)
  - `<OUT_DIR>/meta/step2/...` for manifests and metadata
- `RUN_ID` is used for naming the run manifest (`run=<RUN_ID>.jsonl`).


In [ ]:
from pathlib import Path

# Default path used by the repository fixtures
default_index = PROJECT_ROOT / "out" / "meta" / "step1" / "index.json"

# Or set it manually, e.g.:
# default_index = Path("/abs/path/to/your/step1/root/meta/manifest/index.json")

INDEX_PATH = default_index
print("INDEX_PATH:", INDEX_PATH)
assert INDEX_PATH.exists(), f"index.json does not exist: {INDEX_PATH}"

# DATA_ROOT = dataset root (where /meta/manifest/index.json resides)
DATA_ROOT = INDEX_PATH.parents[2]
print("DATA_ROOT:", DATA_ROOT)

# Step‑2 writes to DATA_ROOT/processed and DATA_ROOT/meta/step2
OUT_DIR = DATA_ROOT

# Run identifier (used to name the run manifest)
RUN_ID = "local"

## 3) Inspect the Step‑1 index and select the first scene

We load the Step‑1 manifest and print the first available `(tile, sensing)` scene.
For real runs you will iterate over all scenes (or filter them by your criteria).


In [ ]:
from s2pipe.preprocess.inputs import load_download_index

index = load_download_index(INDEX_PATH)
print("scenes:", len(index.scenes))
assert len(index.scenes) > 0

scene0 = index.scenes[0]
print("tile:", scene0.tile_id)
print("sensing:", scene0.sensing_start_utc)

## 4) Step‑2 configuration

The configuration below is intentionally lightweight for local testing:

- `max_scenes=1` limits the run to the first scene.
- `target_grid_ref="scl_20m"` uses the `SCL_20m` grid as the target grid for `x.tif` and `y.tif`.
- `to_toa_reflectance=True` converts L1C DN to TOA reflectance per band using `MTD_MSIL1C.xml`.
- Normalization is configured via `NormalizeConfig`; the run itself will choose the mode per pass.

Notes:
- Angles are written to **a separate file** (`angles.tif`) on the coarse native angle grid.
  They are *not* appended into `x.tif`.
- Validity masks are computed **after resampling** (valid=1, nodata=0), based on the L1C convention `DN==0 => nodata`.


In [ ]:
from s2pipe.preprocess.cfg import (
    PreprocessConfig,
    AngleAssetConfig,
    NormalizeConfig,
    LabelConfig,
)

# Angle export is controlled by AngleAssetConfig.enabled (angles are written to a separate angles.tif).
angles_cfg = AngleAssetConfig(
    enabled=True,
    include_sun=True,
    include_view=True,
    encode="sin_cos",
    view_mode="single",
    view_bands=(),
    detector_aggregate="nanmean",
)

# Base normalization config (mode is overridden per-pass)
norm_cfg = NormalizeConfig(
    mode="none",
    stats_path=OUT_DIR / "normalize" / "stats.json",
    clip_percentiles=(1.0, 99.0),
    hist_range=(-0.2, 2.0),
    hist_bin_width=1e-4,
    max_pixels_per_scene=None,
    seed=0,
    save_histograms=False,
)

# Labels config: whether to generate them and what is their source
label_cfg = LabelConfig(
    enabled=False,
    backend="scl",
)

cfg = PreprocessConfig(
    index_json=INDEX_PATH,
    out_dir=OUT_DIR,
    run_id=RUN_ID,
    max_scenes=1,
    target_grid_ref="scl_20m",
    l1c_bands=(
        "B01",
        "B02",
    ),
    to_toa_reflectance=True,
    angles=angles_cfg,
    labels=label_cfg,
    normalize=norm_cfg,
)

cfg

## 5) Run Step‑2 (`run_preprocess`) in two passes

We run preprocessing twice:

**Pass 1 — compute stats**
- `normalize.mode="compute_only"`
- reads tiles, builds per‑band histograms, and writes `stats.json` (and optionally `stats.npz`)
- does **not** export `x.tif`/`y.tif`

**Pass 2 — apply stats**
- `normalize.mode="apply_with_stats"`
- loads `stats.json`, applies clipping + normalization on valid pixels
- exports the preprocessed rasters

Both passes use the same `PreprocessConfig` base, including `to_toa_reflectance=True`.


In [ ]:
from dataclasses import replace
from s2pipe.preprocess.run import run_preprocess

# Pass 1: compute dataset-level normalization stats (writes stats.json and optionally stats.npz)
cfg_stats = replace(
    cfg, normalize=replace(cfg.normalize, mode="compute_only", save_histograms=True)
)
result_stats = run_preprocess(cfg_stats)
# result_stats

# Pass 2: apply stats and export normalized tiles (x.tif/y.tif/optional angles.tif)
cfg_apply = replace(
    cfg,
    normalize=replace(cfg.normalize, mode="apply_with_stats", save_histograms=False),
)
result_apply = run_preprocess(cfg_apply)
result_apply

## 6) Verify outputs and perform a quick inspection

After Pass 2, we load:
- the Step‑2 `index.json`
- the exported `x.tif` and `y.tif`

This is a lightweight sanity check (shapes, dtypes, basic visualization).
For thorough validation you may want to add numerical checks and compare against reference statistics.


In [ ]:
import json
import rasterio

assert result_apply.index_path is not None
step2_index = json.loads(Path(result_apply.index_path).read_text(encoding="utf-8"))
print("Step-2 samples:", len(step2_index.get("samples", [])))

# Find the record for the first scene
sample = None
for s in step2_index.get("samples", []):
    k = s.get("key", {})
    if (
        k.get("tile_id") == scene0.tile_id
        and k.get("sensing_start_utc") == scene0.sensing_start_utc
    ):
        sample = s
        break

assert sample is not None, "Sample not found in meta/step2/index.json"
print("status:", sample.get("status"))
print("paths:", sample.get("paths"))

x_path = OUT_DIR / sample["paths"]["x"]
y_path = OUT_DIR / sample["paths"]["y"] if cfg.labels.enabled else None
m_path = OUT_DIR / sample["paths"]["meta"]
angles_path = (
    OUT_DIR / sample["paths"].get("angles", "") if "angles" in sample["paths"] else None
)

print("x:", x_path)
if y_path is not None:
    print("y:", y_path)
print("meta:", m_path)
print("angles:", angles_path)

assert x_path.exists() and m_path.exists()
if y_path is not None:
    assert y_path.exists()
if angles_path is not None:
    assert angles_path.exists()

meta = json.loads(m_path.read_text(encoding="utf-8"))
print("meta.schema:", meta.get("schema"))
print("meta.key:", meta.get("key"))
print("meta.channels.x:", meta.get("channels", {}).get("x"))
print("meta.channels.y:", meta.get("channels", {}).get("y"))

if angles_path is not None:
    # angles are stored separately (coarse grid)
    assert meta.get("paths", {}).get("angles") == "angles.tif"
    a_info = meta.get("angles", {})
    print("angles.channels:", len(a_info.get("channels", [])))
    print("angles.grid:", a_info.get("grid", {}))

    with rasterio.open(angles_path) as ds:
        a = ds.read()  # (C,H,W)
        print("angles.tif shape:", a.shape, "dtype:", a.dtype)
        assert ds.count == a.shape[0]
        assert ds.width <= 64 and ds.height <= 64  # typically ~23x23

### 6.1) Load and visualize one X channel and Y

The exact number of channels in `x.tif` depends on `valid_pixel_mask`:
- `single`: one mask channel appended at the end
- `per_band`: one mask channel per band appended at the end

The plot below displays the first band and the label raster (`y.tif`) for a quick sanity check.


In [ ]:
import matplotlib.pyplot as plt

with rasterio.open(x_path) as ds:
    x = ds.read()  # (C,H,W)
    print("X shape:", x.shape, "dtype:", x.dtype)

plt.figure()
plt.title("X[0] (e.g., B01)")
plt.imshow(x[0], interpolation="nearest")
plt.colorbar()
plt.show()

if y_path is not None and y_path.exists():
    with rasterio.open(y_path) as ds:
        y = ds.read(1)  # (H,W)
        print("Y shape:", y.shape, "dtype:", y.dtype)

    plt.figure()
    plt.title("Y (labels)")
    plt.imshow(y, interpolation="nearest")
    plt.colorbar()
    plt.show()

if angles_path is not None and angles_path.exists():
    with rasterio.open(angles_path) as ds:
        a = ds.read()  # (C,Hc,Wc)
        print("Angles shape:", a.shape, "dtype:", a.dtype)

    # Visualize the first angles channel for sanity (typically sun_zen_sin)
    plt.figure()
    plt.title("Angles[0] (coarse grid)")
    plt.imshow(a[0], interpolation="nearest")
    plt.colorbar()
    plt.show()

## 7) Recommendations for a real run

- Increase `max_scenes` (or set it to `None`) to process the full dataset.
- Consider setting `NormalizeConfig.max_pixels_per_scene` for faster iterative experiments.
- Keep `save_histograms=True` only when you need to debug or audit histogram shapes; `.npz` can be large.
- Run with angles disabled unless you need them (`angles.enabled=False`) to reduce I/O.
- For production, run from the CLI / pipeline entrypoint and track runs via the generated manifests.
